In [ ]:
import rasterio
import numpy as np
import cv2
import tqdm

fp = r"G:\Mi unidad\Equipo_Agua\Geo\Data\TLALNEPANTLA DE BAZ\Raster\zona_16.tif"
carpeta_salida = r"C:\Users\52551\Documents\Con_lago"
nombre = 'zona_16'
parametro = 1024



img = rasterio.open(fp)
print('Imagen leida.')
array = img.read()
four_images=[array[2],array[1],array[0],array[3]]
stacked_images = np.stack(four_images, axis=-1)
H,W,D=stacked_images.shape
z=0
for i in tqdm.tqdm(range(int(H/parametro))):
    for j in range(int(W/parametro)):
        array1=stacked_images[parametro*i:parametro*(i+1),parametro*j:parametro*(j+1)]
        cv2.imwrite(carpeta_salida + "/" + nombre +'_' +str(i)+"_"+str(j)+".png",array1)
        break

In [4]:
from codigos import particionar_imagen
fp = r"C:\Users\52551\Documents\Trabajo\Equipo_Agua\Geo\Data\Naucalpan\Inclinado.tif"
path_salida = r"G:\Mi unidad\Equipo_Agua\Geo\Data\Naucalpan\Fragmentos"
parametro = 1024

particionar_imagen(fp, parametro=parametro, ruta_salida=path_salida, nombre_salida='Naucalpan_Inclinado_')

Imagen leida.


100%|██████████| 6/6 [00:17<00:00,  2.97s/it]


In [ ]:
def particionar_imagen (fp:str, parametro:int, ruta_salida:str, nombre_salida:str):
    '''
    (Function)
        Esta funciona toma el archivo "fp" y lo particiona en n-elementos (imagenes) para tener fragmentos de la original
        con un zoom siendo cuadradas
    (Paramateres)
        - fp: Ruta de acceso al archivo .tif (Obligatoriamente) 
        - parametro: cantidad de pixeles que tomara para formar un cuadrado (fragmento)
    '''
    img = rasterio.open(fp)
    array = img.read()
    four_images=[array[2],array[1],array[0],array[3]]
    stacked_images = np.stack(four_images, axis=-1)
    H,W,D=stacked_images.shape
    z=0
    for i in tqdm.tqdm(range(int(H/parametro))):
        for j in range(int(W/parametro)):
            array1=stacked_images[parametro*i:parametro*(i+1),parametro*j:parametro*(j+1)]
            cv2.imwrite(ruta_salida+"/"+nombre_salida+str(i)+"_"+str(j)+".png",array1)

In [ ]:

ds = gdal.Open(fp)
width = ds.RasterXSize
height = ds.RasterYSize
gt = ds.GetGeoTransform()
minx = gt[0]
miny = gt[3] + width*gt[4] + height*gt[5] 
maxx = gt[0] + width*gt[1] + height*gt[2]
maxy = gt[3] 

## generar mosaico

In [ ]:
##este si
import rasterio
import cv2
from osgeo import gdal
import numpy as np
import tqdm
import rasterio.mask
output_path=r"H:\Mi unidad\Raster/tlalne/"

In [ ]:
#este si
raster = r"H:\Mi unidad\Raster\zona_18.tif"
gdal_interpeter = gdal.Open(raster)
width = gdal_interpeter.RasterXSize
height = gdal_interpeter.RasterYSize
coordenadas_gdal = gdal_interpeter.GetGeoTransform()
minx = coordenadas_gdal[0]
miny = coordenadas_gdal[3] + width*coordenadas_gdal[4] + height*coordenadas_gdal[5] 
maxx = coordenadas_gdal[0] + width*coordenadas_gdal[1] + height*coordenadas_gdal[2]
maxy = coordenadas_gdal[3] 
minx,maxx,miny,maxy,"W",maxx-minx,"H",maxy-miny

In [ ]:
#este si
src_raster_path = raster
src=rasterio.open(src_raster_path)
H,W=src.shape

In [ ]:
#este si
mode=True ##True para geotiff False para png
dim=1024 ##pixeles
alto=int(np.floor(H/dim))
ancho=int(np.floor(W/dim))
for j in tqdm.tqdm(range(ancho)):#ancho
    for i in (range(alto)):#alto
        # j=1
        label=raster.replace("\\","/").split("/")[-1][:-4]+"_"
        nameimg=label.lower()+str(i)+"_"+str(j)
        cuadro=[]
        for k in range(2): 
            for l in range(2):
                cuadro.append((minx+(maxx-minx)/ancho*(j+k),
                               maxy-(maxy-miny)/alto*(i+l),
                               0.0))
        cuadro=[cuadro[0],cuadro[1],cuadro[3],cuadro[2],cuadro[0]]
        shapes=[{"type":'Polygon','coordinates':[cuadro]}]
        vector=[]
        if mode==False:
            array, out_transform = rasterio.mask.mask(src, shapes, crop=True)
            array=array.copy()
            four_images=[array[2],array[1],array[0]]
            stacked_images = np.stack(four_images, axis=-1)
            imagen_n=0
            imagen_n=stacked_images.copy()
            cv2.imwrite(output_path+nameimg+'.png',imagen_n)
        if mode==True:
            out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True) # setting all pixels outside of the feature zone to zero
            out_meta = src.meta

            out_meta.update({"driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform})

            output_file = output_path+"/"+nameimg+".tif"

            with rasterio.open(output_file, "w", **out_meta) as dest:
                dest.write(out_image)

Versión para quitar calles dentro de capa Raster

In [7]:
import fiona
import rasterio
from rasterio.mask import mask

def create_mask_from_shapefile(shapefile_filepath, corresponding_orthomosaic_filepath):

    # open shapefile
    with fiona.open(shapefile_filepath, 'r') as shapefile:
        shapes = [feature['geometry'] for feature in shapefile]

    # open rasterfile
    with rasterio.open(corresponding_orthomosaic_filepath, 'r') as src:
        out_image, out_transform = mask(src, shapes, crop=True) # setting all pixels outside of the feature zone to zero
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform})

    output_file = r"D:/fer.tif"

    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(out_image)

input_raster = r"C:\Users\ASUS\Desktop\naucalpan.tif"
input_shapefile = r"C:\Users\ASUS\Desktop\qgiszonas\nau_fer.shp"

create_mask_from_shapefile(input_shapefile, input_raster)